In [1]:
import psycopg2
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
from matplotlib import pyplot as plt
warnings.simplefilter("ignore", UserWarning)

In [2]:
#pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [4]:
print(user)

natalia


In [5]:
def connect_to_redshift(dbname, host, port, user, password):
    """Method that connects to redshift. This gives a warning so will look for another solution"""

    connect = psycopg2.connect(
        dbname=dbname, host=host, port=port, user=user, password=password
    )

    print("connection to redshift made")

    return connect

In [6]:
connect=connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [7]:
query="""select otc.*
              
         from bootcamp.online_transactions_cleaned otc   
      
      """

In [8]:
otc=pd.read_sql(query,connect)

# Quering the cleaned data, which were loaded to S3 bucket as a part of ETL pipeline project 


In [9]:
otc.head(10)

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536370,21791,VINTAGE HEADS AND TAILS CARD GAME,1.25,24,30.00,2010-12-01 08:45:00,u12583,France
1,536385,22783,SET 3 WICKER OVAL BASKETS W LIDS,19.95,1,19.95,2010-12-01 09:56:00,u1742,United Kingdom
2,536392,22128,PARTY CONES CANDY ASSORTED,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
3,536404,22469,HEART OF WICKER SMALL,1.65,12,19.80,2010-12-01 11:29:00,u16218,United Kingdom
4,536412,22274,FELTCRAFT DOLL EMILY,2.95,2,5.90,2010-12-01 11:49:00,u1792,United Kingdom
5,536425,22585,PACK OF 6 BIRDY GIFT TAGS,1.25,12,15.00,2010-12-01 12:08:00,u13758,United Kingdom
6,536464,22960,JAM MAKING SET WITH JARS,4.25,1,4.25,2010-12-01 12:23:00,u17968,United Kingdom
7,536520,20754,RETROSPOT RED WASHING UP GLOVES,2.10,1,2.10,2010-12-01 12:43:00,u14729,United Kingdom
8,536526,21135,VICTORIAN METAL POSTCARD SPRING,1.69,16,27.04,2010-12-01 12:58:00,u14001,United Kingdom
9,536381,22411,JUMBO SHOPPER VINTAGE RED PAISLEY,1.95,10,19.50,2010-12-01 09:41:00,u15311,United Kingdom


In [12]:
otc.shape

(399841, 9)

In [13]:
otc.stock_code.nunique()

3679

In [14]:
print("First transaction: {}, Last transaction: {}".format(min(otc.invoice_date),max(otc.invoice_date)))

First transaction: 2010-12-01 08:26:00, Last transaction: 2011-12-09 12:50:00


In [15]:
otc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            399841 non-null  object        
 1   stock_code         399841 non-null  object        
 2   description        399841 non-null  object        
 3   price              399841 non-null  float64       
 4   quantity           399841 non-null  int64         
 5   total_order_value  399841 non-null  float64       
 6   invoice_date       399841 non-null  datetime64[ns]
 7   customer_id        399841 non-null  object        
 8   country            399841 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.5+ MB


In [16]:
otc.describe()

,price,quantity,total_order_value,invoice_date
count,399841.000000,399841.000000,399841.000000,399841
mean,2.952914,12.224359,20.716904,2011-07-10 12:40:36.644566272
min,0.000000,-80995.000000,-168469.600000,2010-12-01 08:26:00
25%,1.250000,2.000000,4.250000,2011-04-06 15:31:00
50%,1.950000,5.000000,11.580000,2011-07-29 15:59:00
75%,3.750000,12.000000,19.500000,2011-10-20 12:03:00
max,1599.260000,80995.000000,168469.600000,2011-12-09 12:50:00
std,7.317593,250.788860,425.471765,NaN


In [17]:
otc.country.nunique()

37

In [18]:
otc.stock_code.nunique()

3679

In [19]:
otc.description.nunique()

3644

In [20]:
otc.customer_id.nunique()

4363

## How many invoices does the table contain?


In [21]:
query="""select count(distinct invoice) as total_num_invoices
              
         from bootcamp.online_transactions_cleaned otc  
      
      """

In [22]:
num_inv=pd.read_sql(query,connect)

In [23]:
num_inv

,total_num_invoices
0,21791


## When was the first and last purchase?


In [10]:
query=""" select 
                 min(invoice_date) as first_purchase,              
                 max(invoice_date) as last_purchase
         from    bootcamp.online_transactions_cleaned otc         
         order   by first_purchase, last_purchase 
                     

"""

In [11]:
first_last_purchase=pd.read_sql(query,connect)

In [12]:
first_last_purchase


,first_purchase,last_purchase
0,2010-12-01 08:26:00,2011-12-09 12:50:00


In [158]:
query=""" with cte as (select invoice,
          description,
          invoice_date as first_purchase,
          sum(quantity) as total_quantity
         from    bootcamp.online_transactions_cleaned otc 
         group by invoice,description,invoice_date
         order by invoice_date)
         
         select invoice,
                description,
                first_purchase
         from cte
         limit 1;

"""

In [148]:
question2=pd.read_sql(query,connect)

In [149]:
question2

,invoice,description,invoice_date,total_quantity
0,536365,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,6
1,536365,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,8
2,536365,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,6
3,536365,SET 7 BABUSHKA NESTING BOXES,2010-12-01 08:26:00,2
4,536365,CREAM HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,6
...,...,...,...,...
394664,581587,CHILDREN'S APRON DOLLY GIRL,2011-12-09 12:50:00,6
394665,581587,PLASTERS IN TIN STRONGMAN,2011-12-09 12:50:00,12
394666,581587,CHILDRENS APRON SPACEBOY DESIGN,2011-12-09 12:50:00,8
394667,581587,PACK OF 20 SPACEBOY NAPKINS,2011-12-09 12:50:00,12


In [159]:
question2=pd.read_sql(query,connect)
question2

,invoice,description,first_purchase
0,536365,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00


In [162]:
query=""" with cte as (select invoice,
          description,
          invoice_date as last_purchase,
          sum(quantity) as total_quantity
         from    bootcamp.online_transactions_cleaned otc 
         group by invoice,description,invoice_date
         order by invoice_date)
         
         select invoice,
                description,
                last_purchase
         from cte
         order by last_purchase desc
         limit 1;

"""

In [163]:
question2=pd.read_sql(query,connect)
question2

,invoice,description,last_purchase
0,581587,ALARM CLOCK BAKELIKE RED,2011-12-09 12:50:00


## How many customers does the table contain?


In [164]:
query=""" select count(distinct customer_id) as numberOfCustomers
          from bootcamp.online_transactions_cleaned otc 
"""

In [165]:
# otc.customer_id.nunique()
question3=pd.read_sql(query,connect)
question3

,numberofcustomers
0,4363


## How many different types of stocks did the customer purchase?


In [179]:
query="""
         select customer_id,
                count(distinct stock_code) as numOfStocks    
         from bootcamp.online_transactions_cleaned otc  
         group by customer_id
         order by numOfStocks desc
      
      """

In [180]:
question4=pd.read_sql(query,connect)
question4

,customer_id,numofstocks
0,u14911,1792
1,u12748,1767
2,u17841,1330
3,u14096,1119
4,u14298,884
...,...,...
4358,u1699,1
4359,u13302,1
4360,u15668,1
4361,u16462,1


In [183]:
# The total number of distinct stock items in the customer purchases
query="""select count(distinct stock_code) as total_number_stocks
from bootcamp.online_transactions_cleaned otc 
"""

In [184]:
question4_2=pd.read_sql(query,connect)
question4_2

,total_number_stocks
0,3679


## What is the most popular stock? (Considering top 10 sold items - How does this differ across markets?)


###  Top 10 sold items

In [44]:
otc.stock_code.nunique()

3679

In [100]:
query="""
         select  
                 description,
                 stock_code as top_item,
                 count(stock_code) as frequency 
         from bootcamp.online_transactions_cleaned otc  
         group by top_item,description
         order by frequency  desc
         limit 10;
         
      
      """

In [101]:
top10f=pd.read_sql(query,connect)
top10f

,description,top_item,frequency
0,CREAM HANGING HEART T-LIGHT HOLDER,85123A,2065
1,REGENCY CAKESTAND 3 TIER,22423,1894
2,JUMBO BAG RED RETROSPOT,85099B,1659
3,PARTY BUNTING,47566,1409
4,ASSORTED COLOUR BIRD ORNAMENT,84879,1405
5,LUNCH BAG RED RETROSPOT,20725,1346
6,SET OF 3 CAKE TINS PANTRY DESIGN,22720,1224
7,POPCORN HOLDER,22197,1110
8,JUMBO BAG VINTAGE DOILEY,23203,1107
9,LUNCH BAG BLACK SKULL.,20727,1099


In [ ]:
# Top 10 sold items aggregating for all countries

In [107]:
query="""
         SELECT description,
                stock_code,
                SUM(quantity) AS total_quantity
         FROM  bootcamp.online_transactions_cleaned
         GROUP BY description, stock_code
         ORDER BY total_quantity DESC
         LIMIT 10;
      
      """

In [108]:
top10m=pd.read_sql(query,connect)
top10m

,description,stock_code,total_quantity
0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,84077,53119
1,POPCORN HOLDER,22197,48689
2,JUMBO BAG RED RETROSPOT,85099B,44963
3,ASSORTED COLOUR BIRD ORNAMENT,84879,35215
4,CREAM HANGING HEART T-LIGHT HOLDER,85123A,34185
5,PACK OF 72 RETROSPOT CAKE CASES,21212,33386
6,RABBIT NIGHT LIGHT,23084,27045
7,MINI PAINT SET VINTAGE,22492,25880
8,PACK OF 12 LONDON TISSUES,22616,25305
9,PACK OF 60 PINK PAISLEY CAKE CASES,21977,24129


## Top 10 sold items across markets

In [ ]:
# Top 10 stock items for each country
query=""" WITH RankedSales AS (
    SELECT 
        stock_code, 
        description,
        country, 
        SUM(quantity) AS total_sold,
        RANK() OVER(PARTITION BY country ORDER BY SUM(quantity) DESC) AS rank
    FROM bootcamp.online_transactions_cleaned
    GROUP BY description,stock_code, country
)
SELECT description,stock_code, country,rank, total_sold
FROM RankedSales
WHERE rank <= 10;
      """

In [303]:

top10mCountryRank=pd.read_sql(query,connect)
top10mCountryRank

,description,stock_code,country,rank,total_sold
0,MINI PAINT SET VINTAGE,22492,Australia,1,2916
1,RABBIT NIGHT LIGHT,23084,Australia,2,1884
2,RED HARMONICA IN BOX,21915,Australia,3,1704
3,RED TOADSTOOL LED NIGHT LIGHT,21731,Australia,4,1344
4,DOLLY GIRL LUNCH BOX,22630,Australia,5,1024
...,...,...,...,...,...
470,PACK OF 12 LONDON TISSUES,22616,United Kingdom,6,24297
471,BROCADE RING PURSE,17003,United Kingdom,7,22672
472,PACK OF 72 RETROSPOT CAKE CASES,21212,United Kingdom,8,22182
473,VICTORIAN GLASS HANGING T-LIGHT,22178,United Kingdom,9,21427


In [290]:
# Top 10 stock items across the countries
query=""" WITH RankedSales AS (
    SELECT 
        stock_code, 
        description,
        country, 
        SUM(quantity) AS total_sold,
        RANK() OVER(PARTITION BY country ORDER BY SUM(quantity) DESC) AS rank
    FROM bootcamp.online_transactions_cleaned
    GROUP BY description,stock_code, country
)
SELECT description,stock_code, country,rank, total_sold
FROM RankedSales
WHERE rank =1
order by total_sold desc
limit 10;

      """

In [291]:
top1CountryRank=pd.read_sql(query,connect)
top1CountryRank

,description,stock_code,country,rank,total_sold
0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,84077,United Kingdom,1,47886
1,RABBIT NIGHT LIGHT,23084,Netherlands,1,4801
2,RABBIT NIGHT LIGHT,23084,France,1,3999
3,RABBIT NIGHT LIGHT,23084,Japan,1,3401
4,MINI PAINT SET VINTAGE,22492,Sweden,1,2916
5,MINI PAINT SET VINTAGE,22492,Australia,1,2916
6,POPCORN HOLDER,22197,EIRE,1,1785
7,ROUND SNACK BOXES SET OF4 WOODLAND,22326,Germany,1,1206
8,CHILDRENS CUTLERY POLKADOT PINK,84997D,Spain,1,1089
9,PLASTERS IN TIN WOODLAND ANIMALS,22554,Switzerland,1,636


## What is the average order value i.e. price * quantity?


In [319]:
query=""" 
          select cast(date_part(year,invoice_date) as int) as invoice_year,
                 cast(date_part(month,invoice_date) as int) as invoice_month,
                 round(sum(total_order_value)/count(distinct invoice),2) as average_oder_value
          from bootcamp.online_transactions_cleaned
          group by invoice_year, invoice_month
          order by invoice_year,invoice_month


""" 

In [320]:
average_oder_value=pd.read_sql(query,connect)
average_oder_value

,invoice_year,invoice_month,average_oder_value
0,2010,12,324.14
1,2011,1,384.96
2,2011,2,367.67
3,2011,3,361.36
4,2011,4,310.40
5,2011,5,359.92
6,2011,6,380.27
7,2011,7,373.45
8,2011,8,407.02
9,2011,9,453.58


## How many Stocks have the Description “Unknown”?

In [249]:
query=""" 
          select count(*) as num_trans_no_Description
          from bootcamp.online_transactions_cleaned
          
          where description = 'Unknown'
          
          
          """ 

In [243]:
otc_final[otc_final.description=="Unknown"]

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
71,536557,22686,Unknown,1.25,1,1.25,2010-12-01 14:41:00,u17841,United Kingdom
633,536408,21705,Unknown,1.65,12,19.80,2010-12-01 11:41:00,u14307,United Kingdom
848,536597,21703,Unknown,0.42,4,1.68,2010-12-01 17:35:00,u18011,United Kingdom
893,536736,21703,Unknown,0.42,192,80.64,2010-12-02 12:59:00,u17381,United Kingdom
912,536500,46000M,Unknown,1.55,10,15.50,2010-12-01 12:35:00,u17377,United Kingdom
...,...,...,...,...,...,...,...,...,...
397793,579106,46000M,Unknown,1.55,6,9.30,2011-11-28 11:42:00,u16705,United Kingdom
397824,579260,23444,Unknown,15.00,1,15.00,2011-11-29 08:41:00,u14618,United Kingdom
397894,579535,23444,Unknown,15.00,1,15.00,2011-11-30 09:50:00,u15414,United Kingdom
398125,580622,23444,Unknown,15.00,1,15.00,2011-12-05 12:09:00,u17006,United Kingdom


In [250]:
no_descr=pd.read_sql(query,connect)
no_descr

,num_trans_no_description
0,1172
